In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
# from mlfs import config
# settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/zohra/mlfs-book


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-09 09:49:20,964 INFO: Initializing external client
2025-11-09 09:49:20,970 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-09 09:49:22,656 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279126


'{"country": "sweden", "city": "stockholm", "street": "sollentuna-e4-haggvik", "aqicn_url": "https://api.waqi.info/feed/@7968", "latitude": 59.33, "longitude": 18.07}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [10]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [20]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,7.0,sweden,stockholm,sollentuna-e4-haggvik,2025-11-09,https://api.waqi.info/feed/@7968


In [21]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [22]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-09,6.50,0.1,11.874544,284.036255,stockholm
1,2025-11-10,6.90,0.0,5.116561,50.710617,stockholm
2,2025-11-11,8.00,0.0,8.217153,208.810699,stockholm
3,2025-11-12,8.60,0.2,21.068459,196.886871,stockholm
4,2025-11-13,5.55,0.0,13.684735,305.362549,stockholm
5,2025-11-14,2.95,0.0,13.854155,294.567169,stockholm
6,2025-11-15,2.25,0.0,12.889810,305.909790,stockholm


In [23]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 356.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [24]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-09 09:53:23,502 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279126/fs/1265733/fg/1668513


Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279126/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739345
         }
       },
       "result": {
         "observed_value": 7.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-09T08:53:23.000501Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

In [25]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-09 09:53:40,243 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279126/fs/1265733/fg/1637958


Uploading Dataframe: 100.00% |█| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279126/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-09 09:53:57,603 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-09 09:54:00,795 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-09 09:54:03,985 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-09 09:55:39,695 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-09 09:55:39,885 INFO: Waiting for log aggregation to finish.
2025-11-09 09:55:48,475 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735319
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-09T08:53:40.000242Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

In [4]:
air_quality_fg_v2 = fs.get_feature_group(
    name='air_quality',
    version=2,
)
air_quality_df_v2=air_quality_fg_v2.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.23s) 


In [5]:
air_quality_df_v2.head()

,date,pm25,country,city,street,url,rolling_mean
0,2021-10-16 00:00:00+00:00,5.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968,9.666667
1,2024-03-14 00:00:00+00:00,41.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968,28.666666
2,2016-09-09 00:00:00+00:00,18.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968,11.333333
3,2024-01-31 00:00:00+00:00,14.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968,26.333334
4,2015-09-08 00:00:00+00:00,5.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968,8.666667


In [7]:
air_quality_df_v2=air_quality_df_v2.sort_values("date", ascending=False)
#air_quality_df_v2.head()
new_rolling_mean=air_quality_df_v2.iloc[0:3]["pm25"].mean()
air_quality_df_v2.iloc[0:3]

,date,pm25,country,city,street,url,rolling_mean
914,2025-11-08 00:00:00+00:00,50.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968,20.333334
487,2025-11-07 00:00:00+00:00,30.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968,12.666667
2233,2025-11-06 00:00:00+00:00,20.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968,9.666667


In [8]:
new_rolling_mean

33.333332

In [13]:
aq_today_df_v2 = aq_today_df.copy()
aq_today_df_v2["rolling_mean"] = new_rolling_mean
aq_today_df_v2

,pm25,country,city,street,date,url,rolling_mean
0,7.0,sweden,stockholm,sollentuna-e4-haggvik,2025-11-09,https://api.waqi.info/feed/@7968,33.333332


In [14]:
aq_today_df_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   pm25          1 non-null      float32       
 1   country       1 non-null      object        
 2   city          1 non-null      object        
 3   street        1 non-null      object        
 4   date          1 non-null      datetime64[ns]
 5   url           1 non-null      object        
 6   rolling_mean  1 non-null      float32       
dtypes: datetime64[ns](1), float32(2), object(4)
memory usage: 180.0+ bytes


In [15]:
air_quality_df_v2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3710 entries, 914 to 1832
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype                  
---  ------        --------------  -----                  
 0   date          3710 non-null   datetime64[us, Etc/UTC]
 1   pm25          3710 non-null   float32                
 2   country       3710 non-null   object                 
 3   city          3710 non-null   object                 
 4   street        3710 non-null   object                 
 5   url           3710 non-null   object                 
 6   rolling_mean  3710 non-null   float32                
dtypes: datetime64[us, Etc/UTC](1), float32(2), object(4)
memory usage: 202.9+ KB


In [16]:
air_quality_fg_v2.insert(aq_today_df_v2)

2025-11-09 09:51:13,434 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279126/fs/1265733/fg/1638003


Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time


Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279126/jobs/named/air_quality_2_offline_fg_materialization/executions


(Job('air_quality_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735329
         }
       },
       "result": {
         "observed_value": 7.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-09T08:51:13.000433Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
